# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.Ms = 200 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel200', 'models','t_dGP_train')

Linesearch     25;  Value 3.058509e+04


partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel30');
models = s.models;


## 1.1 RBCM vs TERBCM

In [5]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 2.43751172, MSLL 1.0672, NLPD 3.8989


### 1.1.4 Find multi q

In [6]:
criterion = 'TERBCM';

In [7]:
printf('Optimizing q\r\n');
iqs = 2.2 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.2052
   2.2026
   2.2007
   2.2084
   2.2084
   2.2051
   2.2034
   2.2026
   2.2097
   2.2076
   2.2007
   2.2059
   2.2076
   2.2062
   2.2015
   2.2059
   2.2006
   2.2077
   2.2079
   2.2001
   2.2050
   2.2074
   2.2090
   2.2086
   2.2090
   2.2085
   2.2099
   2.2047
   2.2021
   2.2044
   2.2045
   2.2014
   2.2062
   2.2027
   2.2062
   2.2023
   2.2065
   2.2056
   2.2075
   2.2021
   2.2064
   2.2082
   2.2070
   2.2095
   2.2030
   2.2050
   2.2019
   2.2046
   2.2004
   2.2016
   2.2025
   2.2022
   2.2009
   2.2012
   2.2061
   2.2050
   2.2024
   2.2022
   2.2030
   2.2030
   2.2007
   2.2057
   2.2066
   2.2010
   2.2065
   2.2016
   2.2059
   2.2040
   2.2069
   2.2033
   2.2031
   2.2073
   2.2082
   2.2069
   2.2022
   2.2086
   2.2058
   2.2081
   2.2054
   2.2066
   2.2074
   2.2050
   2.2004
   2.2038
   2.2054
   2.2010
   2.2066
   2.2076
   2.2080
   2.2033
   2.2067
   2.2004
   2.2018
   2.2019
   2.2045
   2.2098
   2.2083
   2.2080


In [8]:
printf('Optimizing q\r\n');
iqs = 0.4 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   0.40683
   0.40351
   0.40938
   0.40170
   0.40092
   0.40948
   0.40019
   0.40204
   0.40942
   0.40328
   0.40761
   0.40810
   0.40923
   0.40519
   0.40957
   0.40317
   0.40920
   0.40879
   0.40300
   0.40166
   0.40341
   0.40697
   0.40550
   0.40198
   0.40868
   0.40562
   0.40942
   0.40710
   0.40177
   0.40106
   0.40092
   0.40236
   0.40383
   0.40196
   0.40332
   0.40006
   0.40382
   0.40197
   0.40691
   0.40800
   0.40684
   0.40079
   0.40801
   0.40427
   0.40262
   0.40342
   0.40795
   0.40376
   0.40580
   0.40298
   0.40961
   0.40852
   0.40367
   0.40664
   0.40737
   0.40029
   0.40300
   0.40695
   0.40831
   0.40452
   0.40248
   0.40581
   0.40211
   0.40989
   0.40443
   0.40230
   0.40548
   0.40199
   0.40634
   0.40463
   0.40953
   0.40513
   0.40338
   0.40746
   0.40931
   0.40049
   0.40698
   0.40313
   0.40668
   0.40342
   0.40608
   0.40611
   0.40495
   0.40238
   0.40327
   0.40285
   0.40710
   0.40687
   0.40354
 

## 1.2 GRBCM vs TEGRBCM

In [9]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 2.41314554, MSLL -0.0578, NLPD 2.7739


### 1.2.4 Find multi q

In [10]:
criterion = 'TEGRBCM';

In [11]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.0043
   2.0021
   2.0044
   2.0058
   2.0093
   2.0029
   2.0007
   2.0033
   2.0016
   2.0081
   2.0086
   2.0042
   2.0019
   2.0084
   2.0068
   2.0039
   2.0006
   2.0059
   2.0013
   2.0066
   2.0001
   2.0020
   2.0023
   2.0038
   2.0053
   2.0096
   2.0036
   2.0055
   2.0071
   2.0074
   2.0090
   2.0003
   2.0036
   2.0031
   2.0078
   2.0017
   2.0072
   2.0010
   2.0040
   2.0024
   2.0001
   2.0016
   2.0001
   2.0067
   2.0088
   2.0098
   2.0080
   2.0004
   2.0003
   2.0037
   2.0021
   2.0015
   2.0089
   2.0068
   2.0040
   2.0007
   2.0073
   2.0002
   2.0089
   2.0099
   2.0031
   2.0055
   2.0077
   2.0084
   2.0015
   2.0041
   2.0057
   2.0064
   2.0043
   2.0017
   2.0061
   2.0022
   2.0043
   2.0064
   2.0072
   2.0096
   2.0031
   2.0029
   2.0031
   2.0048
   2.0069
   2.0015
   2.0064
   2.0013
   2.0086
   2.0052
   2.0003
   2.0026
   2.0018
   2.0084
   2.0057
   2.0002
   2.0004
   2.0056
   2.0021
   2.0085
   2.0096
   2.0076


In [12]:
printf('Optimizing q\r\n');
iqs = 0.2 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   0.20882
   0.20028
   0.20792
   0.20481
   0.20470
   0.20130
   0.20629
   0.20792
   0.20946
   0.20427
   0.20309
   0.20347
   0.20527
   0.20685
   0.20683
   0.20319
   0.20714
   0.20367
   0.20239
   0.20036
   0.20113
   0.20554
   0.20718
   0.20227
   0.20398
   0.20890
   0.20475
   0.20088
   0.20020
   0.20151
   0.20091
   0.20941
   0.20644
   0.20480
   0.20112
   0.20698
   0.20557
   0.20498
   0.20048
   0.20550
   0.20774
   0.20136
   0.20694
   0.20838
   0.20025
   0.20749
   0.20219
   0.20318
   0.20709
   0.20341
   0.20683
   0.20427
   0.20224
   0.20324
   0.20903
   0.20933
   0.20071
   0.20987
   0.20287
   0.20609
   0.20400
   0.20344
   0.20377
   0.20905
   0.20572
   0.20079
   0.20187
   0.20108
   0.20047
   0.20109
   0.20015
   0.20016
   0.20919
   0.20682
   0.20064
   0.20746
   0.20317
   0.20734
   0.20259
   0.20072
   0.20071
   0.20927
   0.20482
   0.20641
   0.20125
   0.20510
   0.20567
   0.20608
   0.20009
 

## 1.3 GPoE vs TEGPoE

In [13]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 2.09127835, MSLL 1.2658, NLPD 4.0975


### 1.3.4 Find multi q

In [14]:
criterion = 'TEGPoE';

In [15]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
ld = 10
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.0057
   2.0098
   2.0032
   2.0049
   2.0009
   2.0078
   2.0044
   2.0093
   2.0022
   2.0001
   2.0083
   2.0060
   2.0067
   2.0018
   2.0065
   2.0084
   2.0089
   2.0053
   2.0028
   2.0071
   2.0060
   2.0008
   2.0084
   2.0011
   2.0069
   2.0003
   2.0091
   2.0027
   2.0075
   2.0089
   2.0088
   2.0005
   2.0016
   2.0033
   2.0091
   2.0092
   2.0023
   2.0099
   2.0032
   2.0089
   2.0010
   2.0072
   2.0048
   2.0071
   2.0061
   2.0085
   2.0006
   2.0084
   2.0003
   2.0043
   2.0067
   2.0053
   2.0059
   2.0056
   2.0067
   2.0030
   2.0069
   2.0063
   2.0021
   2.0011
   2.0090
   2.0059
   2.0012
   2.0054
   2.0009
   2.0020
   2.0027
   2.0038
   2.0095
   2.0057
   2.0042
   2.0009
   2.0012
   2.0041
   2.0014
   2.0057
   2.0071
   2.0074
   2.0014
   2.0052
   2.0060
   2.0075
   2.0085
   2.0068
   2.0032
   2.0054
   2.0077
   2.0055
   2.0080
   2.0080
   2.0009
   2.0003
   2.0092
   2.0056
   2.0020
   2.0011
   2.0099
   2.0060


In [16]:
printf('Optimizing q\r\n');
iqs = 1.8 + 0.01 * rand(opts.Ms,1)
ld = 10
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   1.8092
   1.8047
   1.8005
   1.8032
   1.8018
   1.8014
   1.8027
   1.8037
   1.8011
   1.8010
   1.8071
   1.8032
   1.8093
   1.8063
   1.8059
   1.8085
   1.8048
   1.8026
   1.8071
   1.8034
   1.8009
   1.8055
   1.8037
   1.8058
   1.8087
   1.8071
   1.8012
   1.8077
   1.8037
   1.8071
   1.8094
   1.8058
   1.8093
   1.8083
   1.8050
   1.8019
   1.8010
   1.8042
   1.8083
   1.8098
   1.8034
   1.8095
   1.8050
   1.8034
   1.8037
   1.8040
   1.8085
   1.8045
   1.8012
   1.8007
   1.8027
   1.8052
   1.8005
   1.8015
   1.8018
   1.8045
   1.8052
   1.8067
   1.8057
   1.8038
   1.8074
   1.8072
   1.8063
   1.8000
   1.8008
   1.8007
   1.8033
   1.8037
   1.8007
   1.8060
   1.8053
   1.8026
   1.8004
   1.8100
   1.8061
   1.8040
   1.8092
   1.8076
   1.8011
   1.8097
   1.8026
   1.8086
   1.8056
   1.8070
   1.8066
   1.8085
   1.8041
   1.8027
   1.8017
   1.8010
   1.8094
   1.8012
   1.8096
   1.8070
   1.8018
   1.8028
   1.8052
   1.8049
